# ✍️ AuraDB — OpenAI RAG (Ejercicio de **consultas**) con LangChain
Cuaderno de **alumno** para consultar un grafo **ya creado** en *Neo4j AuraDB* (IMDB).  
Usaremos **LangChain + OpenAI** para NL→Cypher y responder en español.

**Estructura de cada sección:**  
- Explicación + **Pistas clave**  
- Celda de **código con TODOs** (tú la completas)  
- **### Solución (mostrar/ocultar)** y una celda con la **solución ejecutable**

## ▶️ Instalación rápida
**Docs útiles**
- AuraDB (conectar apps): https://neo4j.com/docs/aura/connecting-applications/overview/  
- Neo4j Python Driver: https://neo4j.com/docs/python-manual/current/  
- LangChain (Neo4j): https://python.langchain.com/docs/integrations/graphs/neo4j  
- Prompt templates: https://python.langchain.com/docs/guides/prompt_templates/

In [ ]:
# Recomendado: ejecutar esta celda primero (reinicia kernel si actualiza mucho)
%pip install -q python-dotenv neo4j langchain langchain-community langchain-openai tiktoken

## 1) Configuración inicial (API Keys + modelo + conexión Neo4j)
**Objetivo:** preparar imports, cargar `.env`, configurar **OpenAI** y crear el conector `Neo4jGraph` (solo lectura).

**Pistas clave**
- `from dotenv import load_dotenv`; `load_dotenv()`  
- Variables: `OPENAI_API_KEY`, `OPENAI_MODEL` (p. ej. `gpt-4.1-mini`)  
- `from langchain_openai import ChatOpenAI` + `os.environ["OPENAI_API_KEY"] = ...`  
- `from langchain_community.graphs import Neo4jGraph` y `Neo4jGraph(url=..., username=..., password=...)`  
- Prueba: `graph.query("RETURN 1 AS ok")`

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY") or "<tu-openai-key>"
OPENAI_MODEL   = os.getenv("OPENAI_MODEL")   or "gpt-4.1-mini"
NEO4J_URI      = os.getenv("NEO4J_URI")      or "neo4j+s://<tu-host>.databases.neo4j.io"
NEO4J_USERNAME = os.getenv("NEO4J_USERNAME") or "neo4j"
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD") or "<tu-contraseña>"

from langchain_openai import ChatOpenAI
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
llm = ChatOpenAI(model=OPENAI_MODEL, temperature=0)

from langchain_community.graphs import Neo4jGraph
graph = Neo4jGraph(url=NEO4J_URI, username=NEO4J_USERNAME, password=NEO4J_PASSWORD)
graph.query("RETURN 1 AS ok")

/var/folders/w8/qlbrzr6d0q92wypgx2t_5vpw0000gn/T/ipykernel_43986/2570950614.py:16: LangChainDeprecationWarning: The class `Neo4jGraph` was deprecated in LangChain 0.3.8 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-neo4j package and should be used instead. To use it run `pip install -U :class:`~langchain-neo4j` and import as `from :class:`~langchain_neo4j import Neo4jGraph``.
  graph = Neo4jGraph(url=NEO4J_URI, username=NEO4J_USERNAME, password=NEO4J_PASSWORD)


[{'ok': 1}]

## 2) Esquema del grafo (solo lectura)
**Objetivo:** refrescar e inspeccionar el esquema para guiar la generación de Cypher.

**Pistas clave**
- `graph.refresh_schema()`  
- `print(graph.schema[:1000])`

In [2]:
# Refresca el esquema del grafo y lo imprime parcialmente para inspección.
graph.refresh_schema()  
# Imprime los primeros 1000 caracteres del esquema del grafo.
# Si el esquema es más largo, añade "..." al final para indicar truncamiento.
print(graph.schema[:1000] + ("..." if len(graph.schema) > 1000 else ""))

Node properties:
Movie {movieId: STRING, runtimeMin: INTEGER, grossUSD: FLOAT, year: INTEGER, description: STRING, imdbRating: FLOAT, votes: INTEGER, metascore: INTEGER, title: STRING}
Year {value: INTEGER}
Decade {value: INTEGER}
RatingBand {name: STRING, min: FLOAT, max: FLOAT}
RuntimeBand {name: STRING, min: INTEGER, max: INTEGER}
BoxOfficeBand {name: STRING, min: FLOAT, max: FLOAT}
Keyword {name: STRING}
Relationship properties:

The relationships:
(:Movie)-[:RELEASED_IN]->(:Year)
(:Movie)-[:HAS_RATING_BAND]->(:RatingBand)
(:Movie)-[:HAS_RUNTIME_BAND]->(:RuntimeBand)
(:Movie)-[:HAS_BOXOFFICE_BAND]->(:BoxOfficeBand)
(:Movie)-[:HAS_KEYWORD]->(:Keyword)
(:Year)-[:IN_DECADE]->(:Decade)


## 3) Prompt de generación de Cypher (solo lectura)
**Objetivo:** construir un `PromptTemplate` que **prohíba escrituras** y use `{schema}` y `{question}`.

**Pistas clave**
- `from langchain.prompts import PromptTemplate`  
- `input_variables=["schema","question"]`  
- Incluir: “NO usar CREATE/MERGE/SET/DELETE/REMOVE/DROP/LOAD CSV”

In [3]:
from langchain.prompts import PromptTemplate
CYTHER_PROMPT_TMPL = PromptTemplate(
    input_variables=["schema","question"],
    template=(
        "Eres un generador de Cypher para Neo4j. "
        "Responde SOLO con una consulta de LECTURA. "
        "NO crear/modificar datos (prohibido CREATE, MERGE, SET, DELETE, REMOVE, DROP, LOAD CSV).\n"
        "Esquema:\n{schema}\n\n"
        "Pregunta: {question}\n"
        "Cypher:"
    ),
)

## 4) Utilidad `clean_fences(text)`
**Objetivo:** si el LLM devuelve ```cypher ...```, quitar fences y etiquetas para quedarnos solo con el Cypher.

**Pistas clave**
- `text.strip()` y `splitlines()`  
- Si empieza por ```, quitar backticks y primera línea si no empieza por `MATCH/CALL/RETURN/WITH`

In [4]:
def clean_fences(text: str) -> str:
    """
    Limpia el texto generado por el LLM eliminando etiquetas de formato y líneas innecesarias.

    Args:
        text (str): Texto que puede contener etiquetas de formato como ```cypher.

    Returns:
        str: Texto limpio, listo para ser interpretado como una consulta Cypher.
    """
    if not text:
        # Si el texto está vacío, se devuelve tal cual.
        return text
    text = text.strip()  # Elimina espacios en blanco al inicio y al final.
    if text.startswith("```"):
        # Si el texto comienza con ``` (etiquetas de bloque de código), las elimina.
        text = text.strip("`")
        lines = text.splitlines()  # Divide el texto en líneas.
        if lines and not lines[0].strip().upper().startswith(("MATCH", "CALL", "RETURN", "WITH")):
            # Si la primera línea no contiene una palabra clave Cypher válida, se elimina.
            lines = lines[1:]
        # Une las líneas restantes en un solo string, eliminando espacios innecesarios.
        text = "\n".join(lines).strip()
    return text

## 5) Validador: **solo lectura**
**Objetivo:** rechazar Cypher con escrituras (CREATE/MERGE/SET/DELETE/REMOVE/DROP/LOAD CSV / CALL dbms...).

**Pistas clave**
- `import re` + `re.compile(r"...", re.IGNORECASE)`  
- Función `is_read_only(cy: str) -> bool` que busca con `.search(...)`

In [5]:
import re

# Define un patrón de expresiones regulares para detectar comandos Cypher que modifican datos.
# Estos incluyen CREATE, MERGE, SET, DELETE, REMOVE, DROP, LOAD CSV y llamadas específicas como CALL dbms.
READ_ONLY_PATTERN = re.compile(
    r"\b(CREATE|MERGE|SET|DELETE|REMOVE|DROP|LOAD\s+CSV|CALL\s+dbms)\b", 
    re.IGNORECASE  # Ignora mayúsculas y minúsculas al buscar coincidencias.
)

def is_read_only(cy: str) -> bool:
    """
    Verifica si una consulta Cypher es de solo lectura.

    Args:
        cy (str): Consulta Cypher a validar.

    Returns:
        bool: True si la consulta es de solo lectura, False si contiene comandos de escritura.
    """
    # Busca coincidencias con el patrón de escritura en la consulta Cypher.
    # Si no hay coincidencias, la consulta es de solo lectura.
    return not bool(READ_ONLY_PATTERN.search(cy or ""))

## 6) Función `generate_cypher(llm, schema, question)`
**Objetivo:** construir el prompt, invocar el LLM, limpiar fences y validar que sea **solo lectura**.

**Pistas clave**
- `prompt = CYTHER_PROMPT_TMPL.format(schema=schema, question=question)`  
- `text = llm.invoke(prompt).content`  
- `cy = clean_fences(text)` y `is_read_only(cy)`

In [6]:
def generate_cypher(llm, schema: str, question: str) -> str:
    """
    Genera una consulta Cypher basada en una pregunta en lenguaje natural y el esquema del grafo.

    Args:
        llm: Modelo de lenguaje utilizado para generar la consulta.
        schema (str): Esquema del grafo que sirve como contexto para la generación.
        question (str): Pregunta en lenguaje natural que debe responderse con Cypher.

    Returns:
        str: Consulta Cypher generada, validada como de solo lectura.

    Raises:
        ValueError: Si la consulta generada contiene comandos de escritura.
    """
    # Formatea el prompt utilizando el esquema y la pregunta proporcionados.
    prompt = CYTHER_PROMPT_TMPL.format(schema=schema, question=question)
    # Invoca el modelo de lenguaje para generar la consulta Cypher.
    text = llm.invoke(prompt).content
    # Limpia el texto generado para eliminar etiquetas de formato y líneas innecesarias.
    cy = clean_fences(text)
    # Valida que la consulta sea de solo lectura.
    if not is_read_only(cy):
        # Lanza un error si la consulta contiene comandos de escritura.
        raise ValueError(f"Consulta no segura (posible escritura):\n{cy}")
    # Devuelve la consulta Cypher validada.
    return cy

## 7) Función `run_cypher(graph, cypher)`
**Objetivo:** ejecutar la consulta de solo lectura y devolver registros (lista de dicts).

**Pistas clave**
- `graph.query(cypher, params or {})`

In [7]:
def run_cypher(graph, cypher: str, params=None):
    """
    Ejecuta una consulta Cypher en el grafo Neo4j y devuelve los resultados.

    Args:
        graph: Objeto de conexión al grafo (Neo4jGraph).
        cypher (str): Consulta Cypher que se desea ejecutar.
        params (dict, opcional): Parámetros adicionales para la consulta Cypher.

    Returns:
        list[dict]: Lista de registros devueltos por la consulta, donde cada registro es un diccionario.
    """
    # Ejecuta la consulta Cypher en el grafo con los parámetros proporcionados (o un diccionario vacío si no hay parámetros).
    return graph.query(cypher, params or {})

## 8) Función `answer_question(question)`
**Objetivo:** generar Cypher, **imprimirlo**, ejecutarlo y resumir con el LLM en español.

**Pistas clave**
- `cy = generate_cypher(llm, graph.schema, question)`  
- `print(cy)` antes de ejecutar  
- `rows = run_cypher(graph, cy)`  
- `PromptTemplate` para redactar respuesta corta a partir de filas

In [ ]:
from langchain.prompts import PromptTemplate as _PT

# Define un template para generar una respuesta basada en la pregunta y los datos obtenidos.
ANSWER_PROMPT_TMPL = _PT(
    input_variables=["question", "rows"],  # Variables que se inyectarán en el template.
    template=(
        "Pregunta: {question}\n"
        "Datos (JSON abreviado): {rows}\n\n"
        "Redacta una respuesta breve y clara en español, sin inventar datos."
    )
)

def answer_question(question: str, max_rows: int = 20, show_cypher: bool = False):
    """
    Genera una consulta Cypher, la ejecuta y utiliza los resultados para redactar una respuesta.

    Args:
        question (str): Pregunta en lenguaje natural que debe responderse.
        max_rows (int): Número máximo de filas a incluir en la respuesta. Por defecto, 20.
        show_cypher (bool): Si es True, imprime la consulta Cypher generada antes de ejecutarla.

    Returns:
        dict: Un diccionario con la consulta Cypher generada, los resultados obtenidos y la respuesta redactada.
    """
    # Genera la consulta Cypher basada en la pregunta y el esquema del grafo.
    cy = generate_cypher(llm, graph.schema, question)
    
    if show_cypher:
        # Imprime la consulta Cypher generada si show_cypher es True.
        print("— Cypher generado —")
        print(cy)
        print("———————")
    
    # Ejecuta la consulta Cypher y obtiene los resultados.
    rows = run_cypher(graph, cy)
    
    # Limita el número de filas a incluir en la respuesta según max_rows.
    short_rows = rows[:max_rows]
    
    # Genera una respuesta en español utilizando el template y los datos obtenidos.
    ans = llm.invoke(ANSWER_PROMPT_TMPL.format(question=question, rows=short_rows)).content
    
    # Devuelve un diccionario con la consulta Cypher, los resultados y la respuesta generada.
    return {"cypher": cy, "rows": rows, "answer": ans}

## 9) Pruebas guiadas
Ejecuta estas pruebas cuando termines los ejercicios.

In [9]:
#  — Prueba NL→Cypher
out = answer_question("Dame el top 5 de películas de la década de 1990 con mayor puntuación IMDb.")
out["answer"]

— Cypher generado —
MATCH (m:Movie)-[:RELEASED_IN]->(y:Year)-[:IN_DECADE]->(d:Decade {value: 1990})
RETURN m.title, m.imdbRating
ORDER BY m.imdbRating DESC
LIMIT 5
———————


"El top 5 de películas de la década de 1990 con mayor puntuación IMDb es:\n\n1. The Shawshank Redemption - 9.3  \n2. Schindler's List - 9.0  \n3. Pulp Fiction - 8.9  \n4. Forrest Gump - 8.8  \n5. Fight Club - 8.8"

In [10]:
#  — Prueba Cypher directo
res = graph.query("""
MATCH (m:Movie)
RETURN m.title AS title, m.imdbRating AS rating
ORDER BY rating DESC, m.votes DESC
LIMIT 5
""")
res

[{'title': 'The Shawshank Redemption', 'rating': 9.3},
 {'title': 'The Godfather', 'rating': 9.2},
 {'title': 'The Dark Knight', 'rating': 9.0},
 {'title': "Schindler's List", 'rating': 9.0},
 {'title': 'The Lord of the Rings: The Return of the King', 'rating': 9.0}]

In [11]:
#  — Prueba NL→Cypher
out = answer_question("Nombre de películas que duran más de 150 minutos?")
out["answer"]

— Cypher generado —
MATCH (m:Movie)
WHERE m.runtimeMin > 150
RETURN m.title AS movieTitle
———————


"Películas que duran más de 150 minutos incluyen: The Godfather, The Lord of the Rings: The Return of the King, Schindler's List, The Godfather: Part II, The Lord of the Rings: The Two Towers, The Lord of the Rings: The Fellowship of the Ring, The Good, the Bad and the Ugly, Interstellar, Seven Samurai, The Green Mile y Cinema Paradiso."

In [12]:
#  — Prueba Cypher directo
res = graph.query("""
MATCH (m:Movie)
WHERE m.runtimeMin > 150
RETURN m.title AS movieTitle
""")
res

[{'movieTitle': 'The Godfather'},
 {'movieTitle': 'The Dark Knight'},
 {'movieTitle': 'The Lord of the Rings: The Return of the King'},
 {'movieTitle': "Schindler's List"},
 {'movieTitle': 'The Godfather: Part II'},
 {'movieTitle': 'Jai Bhim'},
 {'movieTitle': 'Pulp Fiction'},
 {'movieTitle': 'The Lord of the Rings: The Two Towers'},
 {'movieTitle': 'The Lord of the Rings: The Fellowship of the Ring'},
 {'movieTitle': 'The Good, the Bad and the Ugly'},
 {'movieTitle': 'Soorarai Pottru'},
 {'movieTitle': 'Vikram'},
 {'movieTitle': 'Interstellar'},
 {'movieTitle': 'Saving Private Ryan'},
 {'movieTitle': 'The Green Mile'},
 {'movieTitle': 'Seven Samurai'},
 {'movieTitle': 'Sardar Udham'},
 {'movieTitle': 'The Departed'},
 {'movieTitle': 'Gladiator'},
 {'movieTitle': 'Cinema Paradiso'},
 {'movieTitle': 'Once Upon a Time in the West'},
 {'movieTitle': 'Avengers: Endgame'},
 {'movieTitle': 'Django Unchained'},
 {'movieTitle': 'The Dark Knight Rises'},
 {'movieTitle': 'Drishyam 2'},
 {'movieT

In [13]:
#  — Prueba NL→Cypher
out = answer_question("Qué películas fueron creadas en 2016 y cuáles son sus nombres?")
out["answer"]

— Cypher generado —
MATCH (m:Movie)-[:RELEASED_IN]->(y:Year {value: 2016})
RETURN m.title AS movieName
———————


'Las películas creadas en 2016 son: "Your Name.", "Dangal", "A Silent Voice: The Movie", "The Handmaiden", "Hacksaw Ridge", "The Invisible Guest", "La La Land", "Lion", "Zootopia", "Deadpool", "Airlift", "M.S. Dhoni: The Untold Story", "Sing Street", "I, Daniel Blake", "Hidden Figures", "Hunt for the Wilderpeople", "Manchester by the Sea", "Rogue One: A Star Wars Story", "Captain Fantastic" y "Captain America: Civil War".'

In [14]:
#  — Prueba Cypher directo
res = graph.query("""
MATCH (m:Movie)-[:RELEASED_IN]->(y:Year {value: 2016})
RETURN m.title AS movieName
""")
res

[{'movieName': 'Your Name.'},
 {'movieName': 'Dangal'},
 {'movieName': 'A Silent Voice: The Movie'},
 {'movieName': 'The Handmaiden'},
 {'movieName': 'Hacksaw Ridge'},
 {'movieName': 'The Invisible Guest'},
 {'movieName': 'La La Land'},
 {'movieName': 'Lion'},
 {'movieName': 'Zootopia'},
 {'movieName': 'Deadpool'},
 {'movieName': 'Airlift'},
 {'movieName': 'M.S. Dhoni: The Untold Story'},
 {'movieName': 'Sing Street'},
 {'movieName': 'I, Daniel Blake'},
 {'movieName': 'Hidden Figures'},
 {'movieName': 'Hunt for the Wilderpeople'},
 {'movieName': 'Manchester by the Sea'},
 {'movieName': 'Rogue One: A Star Wars Story'},
 {'movieName': 'Captain Fantastic'},
 {'movieName': 'Captain America: Civil War'},
 {'movieName': 'The Salesman'},
 {'movieName': 'Perfect Strangers'},
 {'movieName': 'Udta Punjab'},
 {'movieName': 'Kubo and the Two Strings'}]

In [17]:
#  — Prueba NL→Cypher
out = answer_question("¿Cuál es el año con más películas estrenadas, si empatan dame el año más reciente?")
out["answer"]

— Cypher generado —
MATCH (m:Movie)-[:RELEASED_IN]->(y:Year)
WITH y.value AS year, COUNT(m) AS movieCount
ORDER BY movieCount DESC, year DESC
LIMIT 1
RETURN year, movieCount
———————


'Con los datos proporcionados, el año con más películas estrenadas es 2014, con 29 películas.'

In [19]:
#  — Prueba Cypher directo
res = graph.query("""
MATCH (:Movie)-[:RELEASED_IN]->(y:Year)
RETURN y.value AS year, count(*) AS num
ORDER BY num DESC, year DESC
LIMIT 10;
""")
res

[{'year': 2014, 'num': 29},
 {'year': 2004, 'num': 29},
 {'year': 2009, 'num': 26},
 {'year': 2019, 'num': 25},
 {'year': 2013, 'num': 25},
 {'year': 2006, 'num': 25},
 {'year': 2001, 'num': 25},
 {'year': 2016, 'num': 24},
 {'year': 2007, 'num': 24},
 {'year': 2012, 'num': 23}]

In [26]:
#  — Prueba NL→Cypher
out = answer_question("Quiero que me des un top 3 de peliculas de cada década basándote en el ranking")
out["answer"]

— Cypher generado —
MATCH (m:Movie)-[:RELEASED_IN]->(y:Year)-[:IN_DECADE]->(d:Decade)
WITH d.value AS decade, m
ORDER BY m.imdbRating DESC, m.votes DESC
WITH decade, collect(m)[0..3] AS topMovies
UNWIND topMovies AS movie
RETURN decade, movie.title AS title, movie.imdbRating AS rating, movie.votes AS votes
ORDER BY decade, rating DESC, votes DESC
———————


"Aquí tienes el top 3 de películas por década según el ranking:\n\n1920:\n1. Metropolis (8.3)\n2. The Kid (8.3)\n3. The Passion of Joan of Arc (8.2)\n\n1930:\n1. Modern Times (8.5)\n2. City Lights (8.5)\n3. M (8.3)\n\n1940:\n1. It's a Wonderful Life (8.6)\n2. Casablanca (8.5)\n3. The Great Dictator (8.4)\n\n1950:\n1. 12 Angry Men (9.0)\n2. Seven Samurai (8.6)\n3. Rear Window (8.5)\n\n1960:\n1. The Good, the Bad and the Ugly (8.8)\n2. Harakiri (8.6)\n3. Once Upon a Time in the West (8.5)\n\n1970:\n1. The Godfather (9.2)\n2. The Godfather: Part II (9.0)\n3. One Flew Over the Cuckoo's Nest (8.7)\n\n1980:\n1. Star Wars: Episode V - The Empire Strikes Back (8.7)\n2. Cinema Paradiso (8.5)  \n(No hay tercera película en los datos para esta década)"

In [27]:
#  — Prueba Cypher directo
res = graph.query("""
MATCH (m:Movie)-[:RELEASED_IN]->(:Year)-[:IN_DECADE]->(d:Decade)
WITH d.value AS decade, m
ORDER BY m.imdbRating DESC, m.votes DESC
WITH decade, collect({title:m.title, rating:m.imdbRating, votes:m.votes})[0..3] AS top3
RETURN decade, top3
ORDER BY decade;
""")
res

[{'decade': 1920,
  'top3': [{'title': 'Metropolis', 'votes': 34709, 'rating': 8.3},
   {'title': 'The Kid', 'votes': 34709, 'rating': 8.3},
   {'title': 'The Passion of Joan of Arc', 'votes': 34709, 'rating': 8.2}]},
 {'decade': 1930,
  'top3': [{'title': 'Modern Times', 'votes': 34709, 'rating': 8.5},
   {'title': 'City Lights', 'votes': 34709, 'rating': 8.5},
   {'title': 'M', 'votes': 34709, 'rating': 8.3}]},
 {'decade': 1940,
  'top3': [{'title': "It's a Wonderful Life", 'votes': 34709, 'rating': 8.6},
   {'title': 'Casablanca', 'votes': 34709, 'rating': 8.5},
   {'title': 'The Great Dictator', 'votes': 34709, 'rating': 8.4}]},
 {'decade': 1950,
  'top3': [{'title': '12 Angry Men', 'votes': 34709, 'rating': 9.0},
   {'title': 'Seven Samurai', 'votes': 34709, 'rating': 8.6},
   {'title': 'Rear Window', 'votes': 34709, 'rating': 8.5}]},
 {'decade': 1960,
  'top3': [{'title': 'The Good, the Bad and the Ugly',
    'votes': 34709,
    'rating': 8.8},
   {'title': 'Harakiri', 'votes': 3

In [20]:
#  — Prueba NL→Cypher

## Joyas ocultas
out = answer_question("¿Qué películas tienen los ratings más altos, pero pocos votos?")
out["answer"]

— Cypher generado —
MATCH (m:Movie)
WHERE m.imdbRating IS NOT NULL AND m.votes IS NOT NULL
WITH m
ORDER BY m.imdbRating DESC, m.votes ASC
RETURN m.title AS Title, m.imdbRating AS Rating, m.votes AS Votes
LIMIT 10
———————


'Todas las películas listadas tienen ratings muy altos (entre 8.8 y 9.3), pero todas cuentan con el mismo número de votos (34,709), por lo que no hay películas con pocos votos dentro de estos datos.'

In [21]:
#  — Prueba Cypher directo

## Joyas ocultas
res = graph.query("""
MATCH (m:Movie)
WHERE m.imdbRating >= 8.5 AND m.votes < 20000
RETURN m.title AS title, m.imdbRating AS rating, m.votes AS votes, m.year AS year
ORDER BY rating DESC, votes ASC
LIMIT 15;
""")
res

[]

## 10) Troubleshooting
- `AuthenticationError`: revisa `NEO4J_URI`, `NEO4J_USERNAME`, `NEO4J_PASSWORD` (usa `neo4j+s://` con Aura).  
- `ServiceUnavailable`: problemas de red/URI o instancia parada.  
- **El Cypher no se ve**: aquí se imprime antes de ejecutar (`answer_question(..., show_cypher=True)`).  
- **LLM propone escritura**: tu validador la detecta y lanza error.